<a href="https://colab.research.google.com/github/manoj-nain/AIChamp1/blob/master/CIFAR_Transfer_Learning_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
import scipy as sp
import numpy as np
import pandas as pd
from numpy.random import rand
pd.options.display.max_colwidth = 600

# Scikit Imports
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.model_selection import train_test_split

# Matplot Imports
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
from keras import callbacks
from keras import optimizers
from keras.datasets import cifar10
from keras.engine import Model
from keras.applications import vgg16 as vgg
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

In [4]:
BATCH_SIZE = 32
epoc = 40
NUM_CLASSES = 10
LEARNING_RATE = 1e-4
MOMENTUM = 0.9

In [5]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [6]:
mean = np.mean(X_train,axis=(0,1,2,3))
std = np.std(X_train,axis=(0,1,2,3))
X_train = (X_train-mean)/(std+1e-7)
X_test = (X_test-mean)/(std+1e-7)

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  y_train, 
                                                  test_size=0.15, 
                                                  stratify=np.array(y_train), 
                                                  random_state=42)

In [8]:
Y_train = np_utils.to_categorical(y_train, NUM_CLASSES)
Y_val = np_utils.to_categorical(y_val, NUM_CLASSES)
Y_test = np_utils.to_categorical(y_test, NUM_CLASSES)

In [9]:
X_train.shape

(42500, 32, 32, 3)

In [10]:
import cv2
X_train = np.array([cv2.resize(x,(48,48)) for x in X_train])
X_val = np.array([cv2.resize(x,(48,48)) for x in X_val])
X_test = np.array([cv2.resize(x,(48,48)) for x in X_test])

In [11]:
X_train.shape

(42500, 48, 48, 3)

In [55]:
# X_train = tf.keras.applications.vgg16.preprocess_input(X_train)
# X_val = tf.keras.applications.vgg16.preprocess_input(X_val)
# X_test = tf.keras.applications.vgg16.preprocess_input(X_test)

In [12]:
base_model = vgg.VGG16(weights='imagenet', 
                       include_top=False, 
                       input_shape=(48, 48, 3))

58892288/58889256 [==============================] - 0s 0us/step


In [21]:
last = base_model.get_layer('block3_pool').output
avg = tf.keras.layers.GlobalAveragePooling2D()(last)
cust_cnn = BatchNormalization()(avg)
cust_cnn = Dense(256, activation='relu')(cust_cnn)
cust_cnn = Dense(256, activation='relu')(cust_cnn)
cust_cnn = Dropout(0.5)(cust_cnn)
pred = Dense(10, activation='softmax')(cust_cnn)
model = Model(base_model.input, pred)

In [20]:
for layer in base_model.layers:
     layer.trainable = True

In [15]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 50:
        lrate = 0.0005
    if epoch > 80:
        lrate = 0.0001
    if epoch > 100:
        lrate = 0.00005
    if epoch > 120:
        lrate = 0.00001
    return lrate

In [22]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [60]:

model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)      

In [17]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(X_train)

In [18]:

from keras.callbacks import LearningRateScheduler

In [23]:
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),\
                    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,epochs=140,\
                    verbose=1,validation_data=(X_val,Y_val),callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/140
1328/1328 [==============================] - 41s 31ms/step - loss: 1.4015 - accuracy: 0.4997 - val_loss: 1.6183 - val_accuracy: 0.4931
Epoch 2/140
1328/1328 [==============================] - 41s 31ms/step - loss: 1.0158 - accuracy: 0.6492 - val_loss: 0.9994 - val_accuracy: 0.6479
Epoch 3/140
1328/1328 [==============================] - 40s 30ms/step - loss: 0.8715 - accuracy: 0.7075 - val_loss: 0.8084 - val_accuracy: 0.7237
Epoch 4/140
1328/1328 [==============================] - 40s 30ms/step - loss: 0.7598 - accuracy: 0.7446 - val_loss: 0.6519 - val_accuracy: 0.7801
Epoch 5/140
1328/1328 [==============================] - 41s 31ms/step - loss: 0.6805 - accuracy: 0.7741 - val_loss: 0.6361 - val_accuracy: 0.7909
Epoch 6/140
1328/1328 [==============================] - 41s 31ms/step - loss: 0.6245 - accuracy: 0.7939 - val_loss: 0.7735 - val_accuracy: 0.7611
Epoch 7/140
1328/1328 [==============================] - 40s 30ms/step - loss: 0.5807 - accuracy: 0.8089 - val_loss: 0

KeyboardInterrupt: ignored

In [24]:
test_acc = model.evaluate(X_test, Y_test)

235/235 [==============================] - 2s 7ms/step - loss: 0.3704 - accuracy: 0.9111


In [26]:
import time
t = time.time()

export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras)

./1602311885.h5


In [27]:
!ls

1602311885.h5  sample_data


In [28]:
reloaded = tf.keras.models.load_model(
  export_path_keras, 
  # `custom_objects` tells keras how to load a `hub.KerasLayer`
  custom_objects={'KerasLayer': hub.KerasLayer})

reloaded.summary()

NameError: ignored

In [29]:
predictions = model.predict(X_test)

In [30]:
test_labels = list(y_test.squeeze())
predictions = list(predictions.argmax(axis=1))

In [31]:
label_dict = {0:'airplane',
             1:'automobile',
             2:'bird',
             3:'cat',
             4:'deer',
             5:'dog',
             6:'frog',
             7:'horse',
             8:'ship',
             9:'truck'}